DATA PREP 
1. Imports

In [ ]:
import cv2, os, numpy as np, time, joblib
from tqdm.notebook import tqdm_notebook
from skimage.feature import local_binary_pattern, hog
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold 
# i asked ai about this and it suggested me this type of kfold. 
# i will provide some info abt it in the latter markdown
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

print("test")

test


-> https://aistudio.google.com/prompts/new_chat 
    Why StratifiedKFold instead of standard KFold?

    Both KFold and StratifiedKFold are used for cross-validation. Cross-validation is a technique used during training (especially for hyperparameter tuning with GridSearchCV or RandomizedSearchCV) to get a more reliable estimate of your model's performance than a single train/validation split.

        KFold: This splits the dataset into k contiguous folds (subsets) without shuffling by default. If you shuffle (shuffle=True), it randomly divides the data into k folds. Each fold is roughly the same size.

        StratifiedKFold: This is similar to KFold, but it makes the splits by preserving the percentage of samples for each class. If your overall dataset has 75% class A and 25% class B, StratifiedKFold will ensure that each of the k folds also has approximately 75% class A and 25% class B samples. Each fold is roughly the same size and has a representative class distribution.

        Why StratifiedKFold for classification (especially with imbalance)?

            Reliable Evaluation: In classification problems, especially with imbalanced datasets (like yours, where Fire=1 is the minority class at ~24% in the Kaggle data), a standard KFold might accidentally create one or more folds that have very few or even zero samples from the minority class. If a fold used for validation has no fire images, the model's performance (like Recall or F1-score) on that fold will be undefined or misleadingly high/low.

            Robust Training: Similarly, if a fold used for training has very few fire images, the model might not learn to identify the minority class effectively from that fold.

            StratifiedKFold avoids this by ensuring each fold is a miniature version of the overall dataset's class distribution. This means the performance metric calculated on each fold is a more reliable estimate, and the average metric across all folds (which GridSearchCV uses) is a better overall performance estimate for your hyperparameter tuning.

    In your project's context: Your Kaggle dataset has class imbalance (755 fire, 244 non-fire). Using StratifiedKFold for the cross-validation within GridSearchCV ensures that when GridSearchCV tests different hyperparameter combinations, it gets a stable and realistic F1-score (or whichever scoring metric you choose) for each combination by evaluating on folds that accurately represent the imbalance you expect the model to face. If you used standard KFold, the F1-scores from different folds could vary wildly just due to random chance in how the minority class samples were distributed across folds.



-> https://www.datacamp.com/tutorial/k-fold-cross-validation
Stratified Group K-Fold Cross-Validation 	Primarily classification with grouped data 	Combines stratification and group integrity, ensuring that groups are not split across folds. 	Great for grouped and imbalanced datasets to maintain both class and group integrity.
just before conclusion there is a table for kfold types, i took the info from there

2. Loading & Extracting Features
TODO: pca yi unuttum

In [1]:
def load_prep_img(data_root, target_size, 
    color_spaces=['bgr', 'hsv', 'ycbcr'], normalize=1):
    processed_data = {cs: [] for cs in color_spaces + ['gray']}
    labels = []
    class_dirs = {'fire_images': 1, 'non_fire_images': 0}
    img_extensions = ['.jpg', '.jpeg', '.png']
    
    total_images_processed = 0
    total_images_skipped = 0
    for class_name, label in class_dirs.items():
        class_dir_path = os.path.join(data_root, class_name)

        if not os.path.isdir(class_dir_path): continue
        
        all_files = os.listdir(class_dir_path)
        image_files = [f for f in all_files if os.path.splitext(f)[1].lower() in img_extensions]

        if not image_files: continue
        
        for filename in tqdm(image_files, desc=f"Processing {class_name}"):
            file_path = os.path.join(class_dir_path, filename)
            img_bgr = cv2.imread(file_path)

            if img_bgr is None:
                total_images_skipped += 1
                continue

            img_resized = cv2.resize(img_bgr, target_size, interpolation=cv2.INTER_LINEAR)
            if normalize: img_resized = img_resized.astype(np.float32) / 255.0

            img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
            processed_data['gray'].append(img_gray)

            if 'bgr' in color_spaces:
                 processed_data['bgr'].append(img_resized)
            if 'hsv' in color_spaces:
                img_hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
                processed_data['hsv'].append(img_hsv)
            if 'ycbcr' in color_spaces:
                img_ycbcr = cv2.cvtColor(img_resized, cv2.COLOR_BGR2YCrCb)
                processed_data['ycbcr'].append(img_ycbcr)

            labels.append(label)
            total_images_processed += 1

        print(f"\ntoplam islenen: {total_images_processed} atlanan: {total_images_skipped}")
    
    output_dtype = np.float32 if normalize else np.uint8
    numpy_data = {}
    for cs, data_list in processed_data.items():
        if data_list:
             numpy_data[cs] = np.array(data_list)
        else:
             if cs == 'gray':
                  numpy_data[cs] = np.array([], dtype=output_dtype).reshape(0, target_size[1], target_size[0])
             else:
                  numpy_data[cs] = np.array([], dtype=output_dtype).reshape(0, target_size[1], target_size[0], 3)

    numpy_data['labels'] = np.array(labels)
    return numpy_data    
        
  

In [ ]:
def extract_color_histograms(img_processed, color_space, bins):
    histograms = []
    if color_space == 'hsv':
        if img_processed.dtype == np.float32:
            hist_h = cv2.calcHist([img_processed], [0], None, [bins], [0, 360])
            hist_s = cv2.calcHist([img_processed], [1], None, [bins], [0, 1])
            histograms.extend([hist_h.flatten(), hist_s.flatten()]) 
        elif img_processed.dtype == np.uint8:
            hist_h = cv2.calcHist([img_processed], [0], None, [bins], [0, 180])
            hist_s = cv2.calcHist([img_processed], [1], None, [bins], [0, 256])
            histograms.extend([hist_h.flatten(), hist_s.flatten()]) 
    elif color_space == 'ycbcr':
        if img_processed.dtype == np.float32:
            hist_y = cv2.calcHist([img_processed], [0], None, [bins], [0, 1])
            hist_cb = cv2.calcHist([img_processed], [1], None, [bins], [-0.5, 0.5])
            hist_cr = cv2.calcHist([img_processed], [2], None, [bins], [-0.5, 0.5])
            histograms.extend([hist_y.flatten(), hist_cb.flatten(), hist_cr.flatten()])
        elif img_processed.dtype == np.uint8:
            hist_y = cv2.calcHist([img_processed], [0], None, [bins], [0, 256])
            hist_cb = cv2.calcHist([img_processed], [1], None, [bins], [0, 256])
            hist_cr = cv2.calcHist([img_processed], [2], None, [bins], [0, 256])
            histograms.extend([hist_y.flatten(), hist_cb.flatten(), hist_cr.flatten()])

    if histograms: return np.concatenate(histograms)
    else: return np.array([]) 

def extract_lbp_features(img_gray, radius, n_points, method):
    if n_points is None:
        n_points = 8 * radius
    lbp_image = local_binary_pattern(img_gray, n_points, radius, method=method)
    if method == 'uniform':
        n_bins = n_points + 2
    else:
         n_bins = int(lbp_image.max() + 1)
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=n_bins, range=(0, n_bins))
    lbp_hist = lbp_hist.astype(np.float32)
    if lbp_hist.sum() > 0:
        lbp_hist /= lbp_hist.sum()
    return lbp_hist.flatten()

def extract_hog_features(img_gray, orientations, pixels_per_cell, cells_per_block, block_norm):
    hog_features = hog(img_gray, orientations=orientations,
                       pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block,
                       block_norm=block_norm,
                       visualize=False, feature_vector=True)
    return hog_features.flatten() 

def combine_features(img_dict, feature_params):
    all_features = []
    if 'hsv' in img_dict:
        hsv_hist = extract_color_histograms(img_dict['hsv'], 'hsv', bins=feature_params.get('hist_bins', 100))
        if hsv_hist.size > 0:
            all_features.append(hsv_hist)
    if 'ycbcr' in img_dict:
         ycbcr_hist = extract_color_histograms(img_dict['ycbcr'], 'ycbcr', bins=feature_params.get('hist_bins', 100))
         if ycbcr_hist.size > 0:
              all_features.append(ycbcr_hist)

    if 'gray' in img_dict:
        img_gray_processed = img_dict['gray'] 

        lbp_features = extract_lbp_features(img_gray_processed,
                                            radius=feature_params.get('lbp_radius', 3),
                                            n_points=feature_params.get('lbp_n_points', None),
                                            method=feature_params.get('lbp_method', 'uniform'))
        if lbp_features.size > 0:
             all_features.append(lbp_features)

        hog_features = extract_hog_features(img_gray_processed,
                                           orientations=feature_params.get('hog_orientations', 9),
                                           pixels_per_cell=feature_params.get('hog_pixels_per_cell', (8, 8)),
                                           cells_per_block=feature_params.get('hog_cells_per_block', (2, 2)),
                                           block_norm=feature_params.get('hog_block_norm', 'L2-Hys'))
        if hog_features.size > 0:
             all_features.append(hog_features)

    if all_features:
        combined_vector = np.concatenate(all_features)
        return combined_vector
    else:
        return np.array([])


3. Data Split & Scale

4. Model Training

5. Model Eval